## **IX. Using Advanced Constructs**
Increase your productivity with more advanced coding techniques. You'll learn how to structure data with macros, use conditional-assembly directives to cut down on code repetition, and make your programs more dynamic with control constructs like .IF, .WHILE, and .FOR.  
# Topics covered.
A. How structures differ in assembler

B. Placing code in macros

C. Shortcuts using conditional-assembly directives

1. .IF .ELSE

2. .WHILE

3. .FOR
---

### **A. How structures differ in assembler**

In high-level languages like C or Python, structures (or objects) allow grouping different types of data under one name.
In assembler, you can create **structures** using the `STRUCT` directive, but they don't have built-in member functions — they are simply memory layouts.

**Key Points:**

* A structure in assembler is just a block of memory with named offsets.
* Each field's offset is automatically calculated.
* You access members using the structure name + field name.

#### **Example - Defining and Using a Structure**

In [ ]:
; Example: Defining a POINT structure and using it
; NASM syntax
section .data
    ; Define the structure manually
    struc POINT
        .x: resd 1     ; 4 bytes for X coordinate
        .y: resd 1     ; 4 bytes for Y coordinate
    endstruc

    ; Create an instance of POINT
    myPoint:
        istruc POINT
            at POINT.x, dd 10
            at POINT.y, dd 20
        iend

section .text
    global _start
_start:
    ; Load X coordinate into EAX
    mov eax, [myPoint + POINT.x]
    ; Load Y coordinate into EBX
    mov ebx, [myPoint + POINT.y]

    ; Exit
    mov eax, 60
    xor edi, edi
    syscall

**Explanation:**
We define `POINT` as a structure with `.x` and `.y` fields. Then we create `myPoint` with values 10 and 20. Accessing them is done using `POINT.x` and `POINT.y` offsets.

---

### **B. Placing code in macros**

Macros in assembler are like inline templates — they save time typing repetitive code.

**Key Points:**

* Defined with `%macro` and `%endmacro` in NASM (or `MACRO`/`ENDM` in MASM).
* No function call overhead — code is inserted inline.
* Parameters make them reusable.

#### **Example - Macro for Printing a String (Linux)**

In [ ]:
section .data
    hello db "Hello from macro!", 0xA
    helloLen equ $ - hello

section .text
    global _start

%macro PRINT 2
    mov eax, 1          ; sys_write
    mov edi, 1          ; stdout
    mov rsi, %1         ; string address
    mov rdx, %2         ; string length
    syscall
%endmacro

_start:
    PRINT hello, helloLen
    mov eax, 60         ; exit
    xor edi, edi
    syscall

**Explanation:**
`PRINT` is a macro with two parameters — the string address and length. Instead of writing syscalls manually each time, we just call `PRINT`.

---

### **C. Shortcuts using conditional-assembly directives**

Assembler lets you **conditionally compile code** based on constants, defines, or environment.

#### **1. `.IF / .ELSE`**

In [ ]:
%define DEBUG 1

section .text
global _start
_start:
%if DEBUG = 1
    ; Debug mode code
    nop
%else
    ; Release mode code
    nop
%endif

mov eax, 60
xor edi, edi
syscall

#### **2. `.WHILE`**

```asm
%assign count 0
%while count < 5
    ; This will be repeated at assembly time, not runtime
    %assign count count + 1
%endwhile
```

#### **3. `.FOR`**

```asm
%assign i 0
%rep 3
    ; Assembly-time loop body
    %assign i i + 1
%endrep
```

**Explanation:**
These constructs control **what gets assembled**, not runtime behavior — useful for generating repeated patterns, debug code, or multiple function variations without manual duplication.

⬇️

### **B. Placing Code in Macros**
### **1. Concept Overview**

In assembly language, **macros** are powerful tools that allow you to define a sequence of instructions once and reuse them multiple times. Unlike procedures, macros are expanded **at assembly time**, meaning the assembler replaces each macro call with the actual instructions defined in the macro.

**Key benefits of using macros:**

* **Code reusability** - write once, use anywhere.
* **Parameterization** - macros can accept arguments for flexibility.
* **Performance** - no call/return overhead (since instructions are inserted directly).
* **Readability** - complex operations can be given a meaningful name.

**Key drawbacks:**

* **Increased code size** - because instructions are duplicated wherever the macro is used.
* **No runtime flexibility** - changes require re-assembly.

---

### **2. Syntax for Defining a Macro**

```asm
MyMacro MACRO param1, param2
    mov eax, param1
    add eax, param2
    call DisplayValue
ENDM
```

Here:

* `MACRO` - begins the macro definition.
* `param1, param2` - parameters passed when the macro is used.
* `ENDM` - ends the macro definition.

---

### **3. Example: Using Macros in MASM**

**Code Example - Adding Two Numbers with a Macro**

```asm
; Simple macro to add two values and display the result
INCLUDE Irvine32.inc

AddAndShow MACRO num1, num2
    mov eax, num1
    add eax, num2
    call WriteInt
    call Crlf
ENDM

.data
val1 DWORD 7
val2 DWORD 3

.code
main PROC
    ; Using the macro
    AddAndShow val1, val2
    AddAndShow 100, 250
    exit
main ENDP
END main
```

**Explanation:**

* The `AddAndShow` macro takes **two parameters** (`num1`, `num2`).
* Each call to the macro inserts **mov**, **add**, and display instructions **directly** into the program.
* We demonstrate usage with both **variables** and **immediate values**.

---

### **4. Why Use Macros Instead of Procedures?**

* **When speed is more important than program size** (macros avoid procedure call overhead).
* **For short, frequently used code snippets**.
* **When you want to make assembly code easier to read** without introducing runtime jumps.

---

### **5. Potential Pitfalls**

* Excessive macro use can **bloat program size**.
* Macros **cannot be debugged like normal functions**—you step through expanded code.
* Overusing macros for large logic blocks can make maintenance harder.

---

### **6. Colab Simulation**

While macros are MASM-specific, in Google Colab you can:

1. Write the `.asm` file containing the macro.
2. Assemble and link it using **JWasm** or **MASM** via DOSBox or Wine in Colab.
3. Run the executable to see output.


⬇️

## C. Shortcuts Using Conditional-Assembly Directives**

### **1. Concept Overview**

Conditional-assembly directives in MASM allow you to control **which code is included during assembly**, and they also provide **higher-level syntax** to make assembly programming more readable and concise.
They work at **assembly time**, not runtime.

Three common constructs are:

1. `.IF / .ELSE / .ENDIF` - conditional branching
2. `.WHILE / .ENDW` - loops
3. `.FOR / .ENDF` - counted loops or iteration over ranges

These make MASM look a little more like high-level languages such as C or Pascal.

---

## **2. `.IF / .ELSE / .ENDIF`**

**Purpose:** Conditional branching without manually writing `CMP` and `JMP` instructions.

**Syntax:**

```asm
.IF condition
    ; code if condition true
.ELSE
    ; code if condition false
.ENDIF
```

**Example - Compare Two Numbers**

In [ ]:
INCLUDE Irvine32.inc

.data
num1 DWORD 15
num2 DWORD 10

.code
main PROC
    mov eax, num1
    mov ebx, num2

    .IF eax > ebx
        call WriteString
        db "num1 is greater",0
    .ELSE
        call WriteString
        db "num2 is greater or equal",0
    .ENDIF

    exit
main ENDP
END main

**Explanation:**

* The `.IF` directive checks the condition at **assembly time** and generates the proper CMP/JMP instructions.
* MASM translates `.IF eax > ebx` into `cmp eax, ebx` + `jg` behind the scenes.

---

## **3. `.WHILE / .ENDW`**

**Purpose:** Repeat a block of code while a condition is true.

**Syntax:**

```asm
.WHILE condition
    ; code to repeat
.ENDW
```

**Example - Countdown**

In [ ]:
INCLUDE Irvine32.inc

.data
count DWORD 5

.code
main PROC
    mov eax, count

    .WHILE eax > 0
        call WriteInt
        call Crlf
        dec eax
    .ENDW

    exit
main ENDP
END main

**Explanation:**

* Equivalent to:

  ```asm
  cmp eax, 0
  jle end_loop
  ; body
  jmp loop_start
  ```

---

## **4. `.FOR / .ENDF`**

**Purpose:** Loop with initialization, condition, and increment.

**Syntax:**

```asm
.FOR (initialization, condition, increment)
    ; loop body
.ENDF
```

**Example - Print First 5 Even Numbers**

In [ ]:
INCLUDE Irvine32.inc

.code
main PROC
    .FOR (mov eax, 0, eax < 10, add eax, 2)
        call WriteInt
        call Crlf
    .ENDF
    exit
main ENDP
END main

**Explanation:**

* Works like a `for` loop in C:
  Initialization → Check Condition → Loop Body → Increment → Repeat.

---

## **5. Why These Shortcuts Help**

* **Cleaner code** - no manual jumps for simple conditions.
* **Less error-prone** - MASM generates correct branching logic.
* **Easier to read** - looks more like a high-level language.

---

## **6. Colab Simulation**

In Colab, you'd:

* Write the `.asm` file with `.IF`, `.WHILE`, `.FOR` constructs.
* Assemble with JWasm/MASM inside DOSBox.
* Run and observe how MASM expands them into regular CMP/JMP instructions.